In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
df = pd.read_csv("../output/weather_csv/Gangneung_weather.csv")
df.head()

,Unnamed: 0,날짜,월,일,평균기온,평균습도,평균풍속,일조시간,풍속계높이
0,0,2007-01-01,1,1,6.0,42.0,2.5,5.0,10
1,1,2007-01-02,1,2,5.6,73.8,2.6,1.0,10
2,2,2007-01-03,1,3,4.4,77.3,2.1,0.0,10
3,3,2007-01-04,1,4,4.8,71.3,2.9,3.5,10
4,4,2007-01-05,1,5,4.7,71.6,2.4,0.0,10


In [4]:
df['북위'] = 37.54
df['해발고도'] = 28

In [5]:
df['u_2'] = df['평균풍속']*4.87/np.log(67.8*df['풍속계높이']-5.42)
df['P'] = 101.3*((293-0.0065*df['해발고도'])/293)**5.26
df['Δ'] = df['평균기온'].apply(lambda x:4098*(0.6108*math.exp((17.27*x)/(x+237.3)))/(x+237.3)**2)
df['gamma'] = 0.665*10**(-3)*df['P']
# P
df['A'] = 1+0.34*df['u_2']
# Q
df['B'] = df['Δ']/(df['Δ']+df['gamma']*df['A'])
# R
df['C'] = df['gamma']/(df['Δ']+df['gamma']*df['A'])
# S
df['D'] = 900/(df['평균기온'] + 273)*df['u_2']
df['e_s'] = df['평균기온'].apply(lambda x:0.6108*math.exp((17.27*x)/(x+237.3)))
df['e_a'] = df['평균습도']/100*df['e_s']
df['e_s-e_a'] = df['e_s'] - df['e_a']
df['doi'] = pd.to_datetime(df['날짜']).dt.strftime('%j')
df['doi'] = df['doi'].apply(pd.to_numeric)
df['dr'] = df['doi'].apply(lambda x: 1+0.033*math.cos(2*math.pi/365*x))
df['δ'] = df['doi'].apply(lambda x: 0.409*math.sin(2*math.pi/365*x-1.39))
# df['dr'] = df['doi'].apply(lambda x: 1+0.033*math.cos(2*math.pi/365*x) if x > 181 else 1+0.033*math.cos(2*math.pi/365*x))
# df['δ'] = df['doi'].apply(lambda x: 0.409*math.sin(2*math.pi/365*x-1.39) if x > 181 else 0.409*math.sin(2*math.pi/365*x-1.39))
df['Φ'] = df['북위']*math.pi/180
df['ω_s'] = df['Φ'].apply(lambda x:math.acos(-math.tan(x)*math.tan(x)))
df['Ra'] = 24*60/math.pi*0.082*df['dr']*\
           (df['ω_s']*df['δ'].apply(lambda x: math.sin(x))*
            df['Φ'].apply(lambda x: math.sin(x))+
            df['Φ'].apply(lambda x: math.cos(x))*
            df['δ'].apply(lambda x: math.cos(x))*
            df['ω_s'].apply(lambda x: math.sin(x)))
df['N'] = 24/math.pi*df['ω_s']
df['Rs'] = (0.25+0.5*df['일조시간']/df['N'])*df['Ra']
df['Rso'] = (0.75 + 2 * 10 ** (-5) * df['해발고도']) * df['Ra']
df['Rs/Rso'] = df['Rs'] / df['Rso']
df['R_ns'] = 0.77 * df['Rs']
df['R_nl'] = 4.903*10**(-9)*(df['평균기온']+273.16)**4*(0.34-0.14*df['e_a']**(0.5))*(1.35*df['Rs/Rso']-0.35)
df['Rn'] = df['R_ns'] - df['R_nl']

In [6]:
df

,Unnamed: 0,날짜,월,일,평균기온,평균습도,평균풍속,일조시간,풍속계높이,북위,...,Φ,ω_s,Ra,N,Rs,Rso,Rs/Rso,R_ns,R_nl,Rn
0,0,2007-01-01,1,1,6.0,42.0,2.5,5.0,10,37.54,...,0.655197,2.202468,2.535929,16.82562,1.010778,1.903367,0.531047,0.778299,2.756082,-1.977783
1,1,2007-01-02,1,2,5.6,73.8,2.6,1.0,10,37.54,...,0.655197,2.202468,2.619185,16.82562,0.732630,1.965856,0.372677,0.564125,1.021316,-0.457191
2,2,2007-01-03,1,3,4.4,77.3,2.1,0.0,10,37.54,...,0.655197,2.202468,2.709235,16.82562,0.677309,2.033443,0.333085,0.521528,0.659584,-0.138056
3,3,2007-01-04,1,4,4.8,71.3,2.9,3.5,10,37.54,...,0.655197,2.202468,2.806047,16.82562,0.993363,2.106106,0.471659,0.764890,1.933225,-1.168336
4,4,2007-01-05,1,5,4.7,71.6,2.4,0.0,10,37.54,...,0.655197,2.202468,2.909586,16.82562,0.727397,2.183819,0.333085,0.560095,0.671427,-0.111331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5109,361,2020-12-27,12,27,8.4,43.4,2.0,4.0,10,37.54,...,0.655197,2.202468,2.271376,16.82562,0.837834,1.704804,0.491455,0.645132,2.348738,-1.703606
5110,362,2020-12-28,12,28,8.7,58.8,2.2,8.1,10,37.54,...,0.655197,2.202468,2.327196,16.82562,1.141965,1.746700,0.653784,0.879313,3.727095,-2.847782
5111,363,2020-12-29,12,29,5.4,59.0,2.5,2.3,10,37.54,...,0.655197,2.202468,2.389910,16.82562,0.760823,1.793771,0.424148,0.585834,1.565016,-0.979182
5112,364,2020-12-30,12,30,-5.3,35.0,4.4,6.5,10,37.54,...,0.655197,2.202468,2.459495,16.82562,1.089944,1.845999,0.590436,0.839257,3.237082,-2.397825
